# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [214]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [215]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Import Data

In [216]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [298]:
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units = sfo_data.groupby('year')['housing_units'].mean()
    housing_units_plot = housing_units.hvplot.bar(ylim=[369000, 388000], title ="Housing Units in San Francisco from 2010 to 2016")
    
    
    return housing_units_plot

    # YOUR CODE HERE!

In [299]:
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    
    avg_gross_rent = sfo_data['gross_rent'].groupby(by='year').mean()

    avg_gross_rent_line = avg_gross_rent.hvplot(ylim= [1200, 4600], title = 'Average Gross Rent in San Francisco' )

  
    
    return avg_gross_rent_line

In [300]:
def average_sales_price():
    
    avg_sales_price = sfo_data['sale_price_sqr_foot'].groupby(by='year').mean()

    avg_sales_price_line = avg_sales_price.hvplot(ylim= [330, 720], title = 'Average Sale Price per Square Foot in San Francisco')

    
    return avg_sales_price_line

In [301]:
def average_price_by_neighborhood():
    sfo_data_2 = sfo_data.groupby([sfo_data.index, 'neighborhood']).mean()
    sfo_data_2 = sfo_data_2.reset_index()
    
    plot_avg_price = sfo_data_2.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood')

    return plot_avg_price


In [302]:
def top_most_expensive_neighborhoods():
    
    sfo_most_expensive = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    sfo_most_expensive = sfo_most_expensive.reset_index()
    
    plot_most_expensive = sfo_most_expensive.hvplot.bar(
        x='neighborhood',
        y='sale_price_sqr_foot',
        rot=45,
        ylim=[600,1000],
        title='Top 10 Expensive Neighborhoods'
)

    return plot_most_expensive

In [303]:
def parallel_coordinates():
    
    sfo_most_expensive = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    sfo_most_expensive = sfo_most_expensive.reset_index()
    
    parallel_coordinates = px.parallel_coordinates(
    sfo_most_expensive,
    dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="gross_rent", width=600)

    return parallel_coordinates


In [304]:
def parallel_categories():
    sfo_most_expensive = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False)[:10]
    sfo_most_expensive = sfo_most_expensive.reset_index()
    
    
    parallel_categories =  px.parallel_categories(
    sfo_most_expensive, dimensions =
     ["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"], 
    color = 'gross_rent', width=600)

    return parallel_categories 
    

In [305]:
def neighborhood_map():
    file_path = Path("Data/neighborhoods_coordinates.csv")

    df_neighborhood_locations = pd.read_csv(file_path)
    df_neighborhood_locations.head()

    sfo_avg = sfo_data.groupby(['neighborhood']).mean().reset_index()
    combined_df = pd.concat([df_neighborhood_locations, sfo_avg], axis = 'columns', join ='inner')

    px.set_mapbox_access_token(mapbox_token)

    map_scatter = px.scatter_mapbox(combined_df,
    lat='Lat',
    lon='Lon',
    size='sale_price_sqr_foot',
    color='gross_rent',
    hover_name='Neighborhood',
    title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
    zoom = 11
)

    return map_scatter



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [306]:
# YOUR CODE HERE!
title = "Real Estate Analysis in San Francisco from 2010 to 2016"
welcome = pn.Row (pn.Column(title), neighborhood_map() )
yrly_mkt_analysis = pn.Row(housing_units_per_year , average_gross_rent , average_sales_price)
neighbor_analysis = pn.Column(average_price_by_neighborhood() , top_most_expensive_neighborhoods())   
parallel_plots = pn.Column(parallel_coordinates() , parallel_categories())

dashboard = pn.Tabs (("Welcome", welcome), 
                     ("Yearly Market Analysis", yrly_mkt_analysis),
                     ("Neighborhood Analysis", neighbor_analysis),
                     ("Parallel Plots Analysis", parallel_plots)


)
dashboard

Tabs
    [0] Row
        [0] Column
            [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive09817')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive09899')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive09981')
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Column
        [0] Plotly(Figure)
        [1] Plotly(Figure)

## Serve the Panel Dashboard

In [143]:
dashboard.servable().show()

Launching server at http://localhost:60355
